In [1]:
import torch
import numpy as np
from utils.ply import *
from utils.pc import *

In [2]:
file_name = 'data/datasets/3depn/ShapeNetPointCloud/02691156/10155655850468db78d106ce0a280f87.ply'
points, _, _  = load_ply(file_name)

In [3]:
points.shape

(10000, 3)

In [4]:
plot_3d_point_cloud(points, in_u_sphere=True, show=True, title=file_name, backend='qtagg')

Switching backend from module://matplotlib_inline.backend_inline to qtagg...


<Figure size 500x500 with 1 Axes>

In [4]:
import torch
import torch_geometric 
from torch_geometric.nn.pool import radius_graph
import networkx as nx

In [5]:
graph = torch.from_numpy(points)
edges = radius_graph(graph, r=0.01)

In [6]:
edges

tensor([[ 665, 6872, 9156,  ..., 9995, 7497, 6074],
        [   0,    0,    0,  ..., 9999, 9999, 9999]])

# Algorithm

1. Count all point in point cloud
2. get edge indices from the point cloud radius graph
3. for each edge (u -> v) get source point coordinates (coord of u) and destination coordinates (coord of v)
4. for this coordinates add a segment

In [9]:
plot_3d_point_cloud(graph.numpy(), edges=edges.numpy()[:, :10000], in_u_sphere=True, show=True, title=file_name, backend='qtagg')

Switching backend from QtAgg to qtagg...


<Figure size 500x500 with 1 Axes>

In [20]:
graph[:5]

tensor([[-0.0477,  0.0515,  0.2680],
        [ 0.1966,  0.0100,  0.0117],
        [-0.0117,  0.0940,  0.2773],
        [ 0.0039,  0.0952,  0.2586],
        [-0.0023,  0.0772,  0.3976]])

In [7]:
from model import *

In [8]:
in_features = 3
out_features = 32
hidden_channels = 16

In [9]:
encoder = GraphEncoder(in_channels=in_features, hidden_channels=hidden_channels, out_channels=out_features, num_layers=3)

In [10]:
output = encoder(graph, edges)

In [11]:
output.shape

torch.Size([10000, 32])

In [19]:
output[:5]

tensor([[ 0.0076,  0.0095, -0.0061, -0.0066,  0.0129,  0.0610, -0.1005, -0.0413,
         -0.1347,  0.1177, -0.0741, -0.0651,  0.0705, -0.0749,  0.0851,  0.0449,
         -0.0190,  0.0116,  0.0943, -0.0276, -0.0253,  0.0220,  0.0476,  0.0300,
          0.0237, -0.0840,  0.0566, -0.0527, -0.0355, -0.1026,  0.1106,  0.0944],
        [ 0.0254,  0.0896, -0.0513, -0.1072, -0.1343, -0.0043,  0.0217,  0.0959,
          0.0198, -0.0731,  0.0694,  0.0111, -0.0826,  0.0785, -0.0932, -0.0316,
         -0.0104,  0.0186,  0.0416,  0.0860,  0.1076, -0.0044, -0.0096, -0.0286,
          0.0514,  0.1683, -0.0299, -0.0480, -0.0659,  0.0602, -0.0274,  0.0227],
        [ 0.0035,  0.0310, -0.0048, -0.0346, -0.0233,  0.0498, -0.0966, -0.0143,
         -0.1315,  0.0944, -0.0557, -0.0600,  0.0435, -0.0456,  0.0594,  0.0443,
         -0.0242,  0.0137,  0.1054, -0.0057,  0.0045,  0.0214,  0.0434,  0.0214,
          0.0294, -0.0399,  0.0529, -0.0632, -0.0494, -0.0886,  0.1040,  0.0966],
        [ 0.0030,  0.0330

In [13]:
decoder = GraphDecoder(in_channels=out_features, hidden_channels=hidden_channels, out_channels=in_features, num_layers=3)

In [14]:
decoder_output = decoder(output, edges)

In [15]:
decoder_output.shape

torch.Size([10000, 3])

In [16]:
decoder_output[:10]

tensor([[ 0.1167, -0.0853,  0.0923],
        [-0.0184,  0.0393,  0.0103],
        [ 0.1228, -0.0883,  0.0954],
        [ 0.1005, -0.0711,  0.0784],
        [ 0.1456, -0.0992,  0.1221],
        [ 0.0940, -0.0712,  0.0669],
        [ 0.1486, -0.0915,  0.1334],
        [ 0.0131, -0.0205, -0.0011],
        [-0.0078,  0.0324,  0.0200],
        [ 0.1330, -0.0871,  0.1132]], grad_fn=<SliceBackward0>)